<a href="https://colab.research.google.com/github/jacobnuttall/CS260-creative-1/blob/master/python_neo4j_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook accompanies this (*link coming*) Towards Data Science blog post by Clair J. Sullivan titled "Create a graph database in Neo4j using Python: 
from data cleaning to graph population."  The bulk of what is demonstrated below is described in detail in the blog post, so that is the first place to check for detailed explanations.

All questions or comments should be directed to her at clair.sullivan@neo4j.com or on Twitter at @CJLovesData1.

# Obtaining the data

The first portion of this notebook makes a connection to Kaggle for the purpose of downloading the [arXiv dataset](https://www.kaggle.com/Cornell-University/arxiv).  

In order to run it, you will need to have an API key for Kaggle, which we will use to set up the Kaggle CLI in this notebook.  Instructions for how to do this can be found in [this Medium post](https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a).

The zipped archive of the dataset is approximately 960 MB and unzipped is approximately 2.6 GB.  You can upload it yourself to Google rather than making the connection via the Kaggle CLI as shown below, but this CLI method is significantly faster.

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

## Note
In this next row, you will upload your kaggle.json file associated with your API key to Google, which is used in the following steps.

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jacobnuttall176","key":"79653ffe3e5cf9f4862b1dbdc1c4ae1e"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           6665        196  1.0              
madhurpant/world-deaths-and-causes-1990-2019                    World Deaths and Causes (1990 - 2019)               442KB  2022-11-29 07:09:27           1235         31  1.0              
thedevastator/jobs-dataset-from-glassdoor                       Salary Prediction                                     3MB  2022-11-16 13:52:31           4390        100  1.0              
thedevastator/how-much-sleep-do-americans-really-get        

In [5]:
!kaggle datasets download -d Cornell-University/arxiv

 98% 1.08G/1.10G [00:04<00:00, 269MB/s]
100% 1.10G/1.10G [00:04<00:00, 245MB/s]


In [6]:
!unzip \*.zip && rm *.zip

Archive:  arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


# Prepare the remainder of the Python environment

Note that `tqdm` is optional, but I like to use it so I can tell where we are in the overall population of the dataframe, knowing that the file is approximately 1.79M rows long.

In [7]:
!pip install neo4j
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 174 kB 46.4 MB/s 
  Created wheel for neo4j: filename=neo4j-5.2.1-py3-none-any.whl size=249499 sha256=574e6b70c4acf6545d2a1524c48df5fa303c258610804869bfd6176981d23fb1
  Stored in directory: /root/.cache/pip/wheels/3f/cb/09/5c56a88c1650f9d4fe57e59d9a67a5d77fcef88f55d5c233f1
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
%matplotlib inline

import json
import time

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from neo4j import GraphDatabase

# Create some helper classes and functions

In [9]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [10]:
def get_author_list(line):
    # Cleans author dataframe column, creating a list of authors in the row.
    
    return [e[1] + ' ' + e[0] for e in line]


def get_category_list(line):
    # Cleans category dataframe column, creating a list of categories in the
    # row.
    
    return list(line.split(" "))


def add_categories(categories):
    # Adds category nodes to the Neo4j graph.
  
    query = '''UNWIND $rows AS row
    MERGE (c:Category {category: row.category})
    RETURN count(*) as total
    '''
    
    return conn.query(query, parameters={'rows': categories.to_dict('records')})


def add_authors(rows, batch_size=10000):
    # Adds author nodes to the Neo4j graph as a batch job.

    query = '''UNWIND $rows AS row
               MERGE (:Author {name: row.author})
               RETURN count(*) as total
    '''
    return insert_data(query, rows, batch_size)


def add_papers(rows, batch_size=5000):
    # Adds paper nodes and (:Author)--(:Paper) and (:Paper)--(:Category)
    # relationships to the Neo4j graph as a batch job.  (Note the smaller batch
    # size due to the fact that this function is adding much more data than the
    # add_authors() function.)

    query = '''
    UNWIND $rows as row
    MERGE (p:Paper {id:row.id}) ON CREATE SET p.title = row.title

    // connect categories
    WITH row, p
    UNWIND row.category_list AS category_name
    MATCH (c:Category {category: category_name})
    MERGE (p)-[:IN_CATEGORY]->(c)

    // connect authors
    WITH distinct row, p // reduce cardinality
    UNWIND row.cleaned_authors_list AS author
    MATCH (a:Author {name: author})
    MERGE (a)-[:AUTHORED]->(p)
    RETURN count(distinct p) as total
    '''

    return insert_data(query, rows, batch_size)


def insert_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.

    total = 0
    batch = 0
    start = time.time()
    result = None

    while batch * batch_size < len(rows):

        res = conn.query(query, parameters={'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)

    return result


# Data loading and cleaning

In [11]:
file = "./arxiv-metadata-oai-snapshot.json"

metadata  = []

lines = 1000000    # 100k for testing
with open(file, 'r') as f:
    for line in tqdm(f): 
        metadata.append(json.loads(line))
        lines -= 1
        if lines == 0: break

df = pd.DataFrame(metadata)
df.head()

999999it [00:47, 20997.77it/s]


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [12]:
df['cleaned_authors_list'] = df['authors_parsed'].map(get_author_list)
df['category_list'] = df['categories'].map(get_category_list)
df = df.drop(['submitter', 'authors', 'comments', 'journal-ref', 'doi', 
              'report-no', 'license', 'versions', 'update_date', 'abstract', 
              'authors_parsed', 'categories'], axis=1)
df.head()

,id,title,cleaned_authors_list,category_list
0,0704.0001,Calculation of prompt diphoton production cros...,"[C. Balázs, E. L. Berger, P. M. Nadolsky, C. -...",[hep-ph]
1,0704.0002,Sparsity-certifying Graph Decompositions,"[Ileana Streinu, Louis Theran]","[math.CO, cs.CG]"
2,0704.0003,The evolution of the Earth-Moon system based o...,[Hongjun Pan],[physics.gen-ph]
3,0704.0004,A determinant of Stirling cycle numbers counts...,[David Callan],[math.CO]
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"[Wael Abu-Shammala, Alberto Torchinsky]","[math.CA, math.FA]"


# Prepare :Category nodes

In [13]:
categories = pd.DataFrame(df[['category_list']])
categories.rename(columns={'category_list':'category'}, inplace=True)
categories = categories.explode('category').drop_duplicates(subset=['category'])

categories.size

156

# Prepare :Author nodes

In [14]:
authors = pd.DataFrame(df[['cleaned_authors_list']])

authors.rename(columns={'cleaned_authors_list':'author'}, inplace=True)
authors=authors.explode('author').drop_duplicates(subset=['author'])

authors.size

774342

# Establish connection to Neo4j Sandbox instance

This demonstrationg uses the [Neo4j Sandbox](sandbox.neo4j.com), but a connection to any running Neo4j database will work, including your local machine.  A Sandbox instance does not last longer than about 3 days, so you will need to populate these values with that of your own instance.

In [17]:
conn = Neo4jConnection(uri="bolt://18.212.34.217:7687", user="neo4j", pwd="trouble-combatant-deals")

# Populate the database

This is achieved in three steps.  First, we create all constraints guaranteeing the uniqueness of each node.  Second, we create the each of the nodes themselves.  Finally, we create the relationships between all nodes (`:AUTHORED` and `:IN_CATEGORY`).  

In [18]:
conn.query('CREATE CONSTRAINT papers IF NOT EXISTS ON (p:Paper) ASSERT p.id IS UNIQUE')
conn.query('CREATE CONSTRAINT authors IF NOT EXISTS ON (a:Author) ASSERT a.name IS UNIQUE')
conn.query('CREATE CONSTRAINT categories IF NOT EXISTS ON (c:Category) ASSERT c.category IS UNIQUE')

[]

In [19]:
add_categories(categories)

[<Record total=156>]

In [20]:
add_authors(authors)

{'total': 10000, 'batches': 1, 'time': 41.64620041847229}
{'total': 20000, 'batches': 2, 'time': 52.43684530258179}
{'total': 30000, 'batches': 3, 'time': 60.450549840927124}
{'total': 40000, 'batches': 4, 'time': 64.04177641868591}
{'total': 50000, 'batches': 5, 'time': 67.02721786499023}
{'total': 60000, 'batches': 6, 'time': 67.93783617019653}
{'total': 70000, 'batches': 7, 'time': 68.83630108833313}
{'total': 80000, 'batches': 8, 'time': 69.56097435951233}
{'total': 90000, 'batches': 9, 'time': 70.259024143219}
{'total': 100000, 'batches': 10, 'time': 71.27084398269653}
{'total': 110000, 'batches': 11, 'time': 72.1412763595581}
{'total': 120000, 'batches': 12, 'time': 73.03204441070557}
{'total': 130000, 'batches': 13, 'time': 74.12950539588928}
{'total': 140000, 'batches': 14, 'time': 75.07221841812134}
{'total': 150000, 'batches': 15, 'time': 76.15630626678467}
{'total': 160000, 'batches': 16, 'time': 77.65308022499084}
{'total': 170000, 'batches': 17, 'time': 79.07857775688171}


{'total': 774342, 'batches': 78, 'time': 141.24335718154907}

# Add :Papers nodes and population all relationships

(This might take a few minutes.)

In [ ]:
add_papers(df)

{'total': 5000, 'batches': 1, 'time': 32.88710355758667}
{'total': 10000, 'batches': 2, 'time': 53.16979169845581}
{'total': 15000, 'batches': 3, 'time': 62.09406280517578}
{'total': 20000, 'batches': 4, 'time': 66.1728687286377}
{'total': 25000, 'batches': 5, 'time': 71.30660891532898}
{'total': 30000, 'batches': 6, 'time': 74.40240144729614}
{'total': 35000, 'batches': 7, 'time': 77.69703269004822}
{'total': 40000, 'batches': 8, 'time': 80.98806118965149}
{'total': 45000, 'batches': 9, 'time': 84.49893236160278}
{'total': 50000, 'batches': 10, 'time': 87.37501692771912}
{'total': 55000, 'batches': 11, 'time': 90.18114018440247}
{'total': 60000, 'batches': 12, 'time': 94.18853640556335}
{'total': 65000, 'batches': 13, 'time': 96.89500594139099}
{'total': 70000, 'batches': 14, 'time': 99.8914623260498}
{'total': 75000, 'batches': 15, 'time': 102.68576908111572}
{'total': 80000, 'batches': 16, 'time': 105.79003715515137}
{'total': 85000, 'batches': 17, 'time': 108.50846338272095}
{'tota

# Confirm the database is properly populated with some simple queries

In [ ]:
query_string = '''
MATCH (c:Category) 
RETURN c.category, SIZE(()-[:IN_CATEGORY]->(c)) AS inDegree 
ORDER BY inDegree DESC LIMIT 20
'''

top_cat_df = pd.DataFrame([dict(_) for _ in conn.query(query_string)])
top_cat_df.head(20)

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x=top_cat_df['c.category'], y=top_cat_df['inDegree'])
plt.xlabel('Category Name', fontsize=18)
plt.ylabel('inDegree',fontsize=18)
plt.xticks(rotation='vertical', fontsize=18)
plt.show()

# Alternative method to read graph data from Neo4j back to Python

In [ ]:
result = conn.query(query_string)

for record in result:
    print(record[0], record[1])